In [2]:
import pandas as pd

In [3]:
github_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv"
url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [4]:
len(df)

1830

In [10]:
df = df.iloc[:300]

In [5]:
df.head()

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


In [6]:
from sentence_transformers import SentenceTransformer
model_name = 'multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(model_name)

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [7]:
answer_llm = df.iloc[0].answer_llm

## Q1

In [18]:
embedding_model.encode(answer_llm)[0]

-0.42244655

## Q2

In [14]:
def compute_similarity(row):
    v_llm = embedding_model.encode(row['answer_llm'])
    
    v_orig = embedding_model.encode(row['answer_orig'])
    multiplied_embeddings = v_llm.dot(v_orig)
    return multiplied_embeddings

In [15]:
evaluations = df.apply(compute_similarity, axis=1).tolist()

In [17]:
df['similarity'] = evaluations
df['similarity'].describe()

count    300.000000
mean      27.495996
std        6.384742
min        4.547924
25%       24.307844
50%       28.336870
75%       31.674309
max       39.476013
Name: similarity, dtype: float64

## Q3

In [22]:
import numpy as np

def normalize(v):
    norm = np.sqrt((v * v).sum())
    v_norm = v / norm
    return v_norm

In [23]:
def compute_similarity_norm(row):
    v_llm = embedding_model.encode(row['answer_llm'])
    v_llm = normalize(v_llm)
    v_orig = embedding_model.encode(row['answer_orig'])
    v_orig = normalize(v_orig)
    multiplied_embeddings = v_llm.dot(v_orig)
    return multiplied_embeddings

In [24]:
df['similarity_norm'] = df.apply(compute_similarity_norm, axis=1).tolist()
df['similarity_norm'].describe()

count    300.000000
mean       0.728393
std        0.157755
min        0.125357
25%        0.651273
50%        0.763761
75%        0.836235
max        0.958796
Name: similarity_norm, dtype: float64

## Q4

In [25]:
!pip install rouge

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [26]:
r = df.iloc[10]

In [29]:
r['document']

'5170565b'

In [30]:
from rouge import Rouge
rouge_scorer = Rouge()

scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]

In [31]:
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

## Q5

In [34]:
(scores['rouge-1']['f']+scores['rouge-2']['f']+scores['rouge-l']['f'])/3

0.35490034990035496

## Q6

In [35]:
def compute_rouge_scores(row):
    scores = rouge_scorer.get_scores(row['answer_llm'], row['answer_orig'])[0]
    return {
        'rouge-1': scores['rouge-1']['f'],
        'rouge-2': scores['rouge-2']['f'],
        'rouge-l': scores['rouge-l']['f']
    }

In [38]:
# Apply the function to each row and create a new DataFrame with the scores
scores_df = df.apply(compute_rouge_scores, axis=1, result_type='expand')

scores_df

,rouge-1,rouge-2,rouge-l
0,0.095238,0.028169,0.095238
1,0.125000,0.055556,0.093750
2,0.415584,0.177778,0.389610
3,0.216216,0.047059,0.189189
4,0.142076,0.033898,0.120219
...,...,...,...
295,0.654545,0.540984,0.618182
296,0.590164,0.460432,0.557377
297,0.654867,0.564516,0.637168
298,0.304762,0.132231,0.304762


In [39]:
scores_df['rouge-2'].describe()

count    300.000000
mean       0.206965
std        0.153550
min        0.000000
25%        0.097809
50%        0.178671
75%        0.286181
max        0.739130
Name: rouge-2, dtype: float64